<a href="https://colab.research.google.com/github/OraMoyal26/bacteria_dimensions/blob/main/Bacteria_FasterRCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2022 The TensorFlow Authors.

### data needed for this code:
### 1. images for training, validation and test(JPG)
### 2. annotation files for training, validation and test(JSON)

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

## Install necessary dependencies

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install -U -q "tensorflow>=2.9.2" "tf-models-official"

In [ ]:
!pip install numpy==1.23

## Import required libraries

In [ ]:
import os
import io
import pprint
import tempfile
import matplotlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd

import cv2

import glob

from PIL import Image
from six import BytesIO
from IPython import display
from urllib.request import urlopen

## Import required libraries from tensorflow models

In [ ]:
import orbit
import tensorflow_models as tfm

from official.core import exp_factory
from official.core import config_definitions as cfg
from official.vision.serving import export_saved_model_lib
from official.vision.ops.preprocess_ops import normalize_image
from official.vision.ops.preprocess_ops import resize_and_crop_image
from official.vision.utils.object_detection import visualization_utils
from official.vision.dataloaders.tf_example_decoder import TfExampleDecoder

pp = pprint.PrettyPrinter(indent=4) # Set Pretty Print Indentation
print(tf.__version__) # Check the version of tensorflow used

%matplotlib inline

### clone the model-garden repo as the required data conversion codes are within this model-garden repository

In [ ]:
!git clone --quiet https://github.com/tensorflow/models.git

### Change directory to vision or data where data conversion tools are available

In [ ]:
%cd ./models/

## install TensorFlow's Object Detection API

In [ ]:
# Install the Object Detection API from GitHub
!git clone https://github.com/tensorflow/models.git
%cd models/research/
!protoc object_detection/protos/*.proto --python_out=.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

# Or install directly via pip if the package is available
!pip install tf-models-official


### CLI command to convert data.

In [ ]:
import os
import subprocess

# Generate TFRecord files for the training path
!python -m official.vision.data.create_coco_tf_record --logtostderr \
  --image_dir='/content/gdrive/MyDrive/bacteria cells/trainjson/train' \ #path to images directory for training
  --object_annotations_file='/content/gdrive/MyDrive/bacteria cells/trainjson/_annotations.coco.json' \ #path to JSON file for training
  --output_file_prefix='/content/gdrive/MyDrive/bacteria cells/trainjson/TFtrain-00000-of-00001.tfrecord' \ #path to save the tfrecord file
  --num_shards=1

# Generate TFRecord files for the validation path
!python -m official.vision.data.create_coco_tf_record --logtostderr \
  --image_dir='/content/gdrive/MyDrive/bacteria cells/val json/valid' \ #path to images directory for validation
  --object_annotations_file='/content/gdrive/MyDrive/bacteria cells/val json/valid/_annotations.coco.json' \ ##path to JSON file for validation
  --output_file_prefix='/content/gdrive/MyDrive/bacteria cells/val json/TFvalid-00000-of-00001.tfrecord' \ #path to save the tfrecord file
  --num_shards=1

# Generate TFRecord files for the test path
!python -m official.vision.data.create_coco_tf_record --logtostderr \
  --image_dir='/content/gdrive/MyDrive/bacteria cells/test json/test' \ #path to images directory for test
  --object_annotations_file='/content/gdrive/MyDrive/bacteria cells/test json/test/_annotations.coco.json' \ ##path to JSON file for test
  --output_file_prefix='/content/gdrive/MyDrive/bacteria cells/test json/TFtest-00000-of-00001.tfrecord' \ #path to save the tfrecord file
  --num_shards=1


## Configure the Retinanet Resnet FPN COCO model for custom dataset.


In [ ]:
train_data_input_path = '/content/gdrive/MyDrive/bacteria cells/trainjson/TFtrain-00000-of-00001.tfrecord'
valid_data_input_path = '/content/gdrive/MyDrive/bacteria cells/val json/TFvalid-00000-of-00001.tfrecord'
test_data_input_path = '/content/gdrive/MyDrive/bacteria cells/test json/TFtest-00000-of-00001.tfrecord'
model_dir = '/content/gdrive/MyDrive/bacteria cells/exported model' #path to the saved model
export_dir ='/content/gdrive/MyDrive/bacteria cells/exported model'

In Model Garden, the collections of parameters that define a model are called *configs*. Model Garden can create a config based on a known set of parameters via a [factory](https://en.wikipedia.org/wiki/Factory_method_pattern).


Use the `retinanet_resnetfpn_coco` experiment configuration, as defined by `tfm.vision.configs.retinanet.retinanet_resnetfpn_coco`.

The configuration defines an experiment to train a RetinanNet with Resnet-50 as backbone, FPN as decoder. Default Configuration is trained on [COCO](https://cocodataset.org/) train2017 and evaluated on [COCO](https://cocodataset.org/) val2017.

There are also other alternative experiments available such as
`retinanet_resnetfpn_coco`, `retinanet_spinenet_coco`, `fasterrcnn_resnetfpn_coco` and more. One can switch to them by changing the experiment name argument to the `get_exp_config` function.

We are going to fine tune the Resnet-50 backbone checkpoint which is already present in the default configuration.

In [ ]:
exp_config = exp_factory.get_exp_config('fasterrcnn_resnetfpn_coco')

### Adjust the model and dataset configurations so that it works with custom

In [ ]:
batch_size = 8
num_classes = 1

HEIGHT, WIDTH = 640, 640
IMG_SIZE = [HEIGHT, WIDTH, 3]




# Backbone config.
exp_config.task.freeze_backbone = False
exp_config.task.annotation_file = ''

# Model config.
exp_config.task.model.input_size = IMG_SIZE
exp_config.task.model.num_classes = num_classes + 1
#exp_config.task.model.detection_generator.tflite_post_processing.max_classes_per_detection = exp_config.task.model.num_classes

# Training data config.
exp_config.task.train_data.input_path = train_data_input_path
exp_config.task.train_data.dtype = 'float32'
exp_config.task.train_data.global_batch_size = batch_size
exp_config.task.train_data.parser.aug_scale_max = 1.0
exp_config.task.train_data.parser.aug_scale_min = 1.0

# Validation data config.
exp_config.task.validation_data.input_path = valid_data_input_path
exp_config.task.validation_data.dtype = 'float32'
exp_config.task.validation_data.global_batch_size = batch_size

### Adjust the trainer configuration.

In [ ]:
logical_device_names = [logical_device.name for logical_device in tf.config.list_logical_devices()]

if 'GPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'GPU'
elif 'TPU' in ''.join(logical_device_names):
  print('This may be broken in Colab.')
  device = 'TPU'
else:
  print('Running on CPU is slow, so only train for a few steps.')
  device = 'CPU'


train_steps = 1100
exp_config.trainer.steps_per_loop = 100  # steps_per_loop = num_of_training_examples // train_batch_size

exp_config.trainer.summary_interval = 100
exp_config.trainer.checkpoint_interval = 100
exp_config.trainer.validation_interval = 100
exp_config.trainer.validation_steps =  34 # validation_steps = num_of_validation_examples // eval_batch_size
exp_config.trainer.train_steps = train_steps
exp_config.trainer.optimizer_config.warmup.linear.warmup_steps = 100
exp_config.trainer.optimizer_config.learning_rate.type = 'cosine'
exp_config.trainer.optimizer_config.learning_rate.cosine.decay_steps = train_steps
exp_config.trainer.optimizer_config.learning_rate.cosine.initial_learning_rate = 0.1
exp_config.trainer.optimizer_config.warmup.linear.warmup_learning_rate = 0.05

### Print the modified configuration.

In [ ]:
pp.pprint(exp_config.as_dict())
display.Javascript('google.colab.output.setIframeHeight("500px");')

### Set up the distribution strategy.

In [ ]:
if exp_config.runtime.mixed_precision_dtype == tf.float16:
    tf.keras.mixed_precision.set_global_policy('mixed_float16')

if 'GPU' in ''.join(logical_device_names):
  distribution_strategy = tf.distribute.MirroredStrategy()
elif 'TPU' in ''.join(logical_device_names):
  tf.tpu.experimental.initialize_tpu_system()
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='/device:TPU_SYSTEM:0')
  distribution_strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
  print('Warning: this will be really slow.')
  distribution_strategy = tf.distribute.OneDeviceStrategy(logical_device_names[0])

print('Done')

## Create the `Task` object (`tfm.core.base_task.Task`) from the `config_definitions.TaskConfig`.

The `Task` object has all the methods necessary for building the dataset, building the model, and running training & evaluation. These methods are driven by `tfm.core.train_lib.run_experiment`.

In [ ]:
with distribution_strategy.scope():
  task = tfm.core.task_factory.get_task(exp_config.task, logging_dir=model_dir)

## Visualize a batch of the data.

In [ ]:
for images, labels in task.build_inputs(exp_config.task.train_data).take(1):
  print()
  print(f'images.shape: {str(images.shape):16}  images.dtype: {images.dtype!r}')
  print(f'labels.keys: {labels.keys()}')

### Create category index dictionary to map the labels to coressponding label names.

In [ ]:
category_index={
    1: {
        'id': 1,
        'name': 'cells'
       }
}

tf_ex_decoder = TfExampleDecoder()

### Helper function for visualizing the results from TFRecords.
Use `visualize_boxes_and_labels_on_image_array` from `visualization_utils` to draw boudning boxes on the image.

In [ ]:
def show_batch(raw_records, num_of_examples):
  plt.figure(figsize=(20, 20))
  use_normalized_coordinates=True
  min_score_thresh = 0.30
  for i, serialized_example in enumerate(raw_records):
    plt.subplot(8, 8, i + 1)
    decoded_tensors = tf_ex_decoder.decode(serialized_example)
    image = decoded_tensors['image'].numpy().astype('uint8')
    scores = np.ones(shape=(len(decoded_tensors['groundtruth_boxes'])))
    visualization_utils.visualize_boxes_and_labels_on_image_array(
        image,
        decoded_tensors['groundtruth_boxes'].numpy(),
        decoded_tensors['groundtruth_classes'].numpy().astype('int'),
        scores,
        category_index=category_index,
        use_normalized_coordinates=use_normalized_coordinates,
        max_boxes_to_draw=200,
        min_score_thresh=min_score_thresh,
        agnostic_mode=True,
        instance_masks=None,
        line_thickness=1)

    plt.imshow(image)
    plt.axis('off')
    plt.title(f'Image-{i+1}')
  plt.show()

### Visualization of train data

The bounding box detection has Percentage of match between predicted and ground truth bounding boxes.

**Note**: The reason of everything is 100% is because we are visualising the groundtruth.

In [ ]:
buffer_size = 20
num_of_examples = 5

raw_records = tf.data.TFRecordDataset(
    exp_config.task.train_data.input_path).shuffle(
        buffer_size=buffer_size).take(num_of_examples)
show_batch(raw_records, num_of_examples)

## Train and evaluate.

We follow the COCO challenge tradition to evaluate the accuracy of object detection based on mAP(mean Average Precision). Please check [here](https://cocodataset.org/#detection-eval) for detail explanation of how evaluation metrics for detection task is done.

**IoU**: is defined as the area of the intersection divided by the area of the union of a predicted bounding box and ground truth bounding box.

In [ ]:
model, eval_logs = tfm.core.train_lib.run_experiment(
    distribution_strategy=distribution_strategy,
    task=task,
    mode='train_and_eval',
    params=exp_config,
    model_dir=model_dir,
    run_post_eval=True)

## Load logs in tensorboard.




In [ ]:
%load_ext tensorboard
%tensorboard --logdir '/content/gdrive/MyDrive/bacteria cells/exported model'


In [ ]:
model.summary()

## Saving and exporting the trained model.

The `keras.Model` object returned by `train_lib.run_experiment` expects the data to be normalized by the dataset loader using the same mean and variance statiscics in `preprocess_ops.normalize_image(image, offset=MEAN_RGB, scale=STDDEV_RGB)`. This export function handles those details, so you can pass `tf.uint8` images and get the correct results.

In [ ]:
export_saved_model_lib.export_inference_graph(
    input_type='image_tensor',
    batch_size=1,
    input_image_size=[HEIGHT, WIDTH],
    params=exp_config,
    checkpoint_path=tf.train.latest_checkpoint(model_dir),
    export_dir=export_dir)


## Inference from trained model

In [ ]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)



def build_inputs_for_object_detection(image, input_image_size):
  """Builds Object Detection model inputs for serving."""
  image, _ = resize_and_crop_image(
      image,
      input_image_size,
      padded_size=input_image_size,
      aug_scale_min=1.0,
      aug_scale_max=1.0)
  return image

### Visualize test data.

In [ ]:
num_of_examples = 10

test_ds = tf.data.TFRecordDataset(
    test_data_input_path).take(
        num_of_examples)
show_batch(test_ds, num_of_examples)

### Importing SavedModel.

In [ ]:
imported = tf.saved_model.load(export_dir)
model_fn = imported.signatures['serving_default']

### Visualize predictions.

In [ ]:

input_image_size = (HEIGHT, WIDTH)
plt.figure(figsize=(20, 20))
min_score_thresh = 0.5 # Change minimum score for threshold to see all bounding boxes confidences.



for i, serialized_example in enumerate(test_ds):
  plt.subplot(5, 5, i+1)

  decoded_tensors = tf_ex_decoder.decode(serialized_example)
  image = build_inputs_for_object_detection(decoded_tensors['image'], input_image_size)
  image = tf.expand_dims(image, axis=0)
  image = tf.cast(image, dtype = tf.uint8)
  image_np = image[0].numpy()
  result = model_fn(image)

  #find the image name
  eager_tensor = serialized_example
  numpy_array = eager_tensor.numpy()
  string = numpy_array.decode('ISO-8859-1')
  file_name=""
  lines = string.splitlines()
  for line in lines:
      if "Box" in line:
          file_name=line
          print(line)



  visualization_utils.visualize_boxes_and_labels_on_image_array(
      image_np,
      result['detection_boxes'][0].numpy(),
      result['detection_classes'][0].numpy().astype(int),
      result['detection_scores'][0].numpy(),
      category_index=category_index,
      use_normalized_coordinates=False,
      max_boxes_to_draw=200,
      min_score_thresh=min_score_thresh,
      agnostic_mode=True,
      instance_masks=None,
      line_thickness=1)
  pic = Image.fromarray(image_np)

  #To save the image with the tags
  pic.save('/content/gdrive/MyDrive/bacteria cells/'+file_name[1:])

  plt.imshow(image_np)
  plt.axis('off')



plt.show()